In [1]:
import numpy as np
import pandas as pd

In [2]:
import yfinance as yf

df = yf.download('INTC', start="2016-01-01")
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-01-04,33.880001,34.009998,33.459999,33.990002,27.142990,27882200
2016-01-05,33.959999,34.000000,33.529999,33.830002,27.015226,16709500
2016-01-06,33.250000,33.520000,32.799999,33.080002,26.416302,25491300
2016-01-07,32.279999,33.009998,31.840000,31.840000,25.426088,37680500
2016-01-08,32.090000,32.220001,31.430000,31.510000,25.162560,29953800
...,...,...,...,...,...,...
2024-04-04,40.590000,40.779999,39.700001,39.730000,39.730000,57307200
2024-04-05,39.349998,39.459999,38.509998,38.709999,38.709999,67189100
2024-04-08,38.639999,38.860001,37.910000,37.980000,37.980000,56967800


In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df[['Close']].values)

In [4]:
input_length = 100
output_length = 10
n_features = 1

X, Y = [], []
for i in range(len(scaled_df) - input_length - output_length + 1):
    X.append(scaled_df[i:i+input_length])
    Y.append(scaled_df[i+input_length:i+input_length+output_length])

X = np.array(X)
Y = np.array(Y)

In [5]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Dropout, LSTM, Flatten, Dense

n_filters = 32
kernel_size = 3
pool_size=2
n_hidden_units = 50

# CNN-LSTM model
model = Sequential()

model.add(Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu', input_shape=(input_length, n_features)))
model.add(MaxPool1D(pool_size=pool_size))
model.add(Dropout(0.2))
model.add(LSTM(units=n_hidden_units, activation='tanh', input_shape=(input_length, n_features)))
model.add(Flatten())
model.add(Dense(output_length))

model.compile(optimizer='adam', loss='mse')

c:\Users\j584l\Downloads\Projects\stock-price-forecasting\env1\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
c:\Users\j584l\Downloads\Projects\stock-price-forecasting\env1\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
from keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X, Y, epochs=100, validation_split=0.2, callbacks=[callback])

Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.1655 - val_loss: 0.0069
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0070 - val_loss: 0.0049
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0063 - val_loss: 0.0053
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0059 - val_loss: 0.0042
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0059 - val_loss: 0.0044
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0055 - val_loss: 0.0042
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0057 - val_loss: 0.0037
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0057 - val_loss: 0.0040
Epoch 9/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0052 - val_loss: 0.0037
Epoch 10/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0053 - val_loss: 0.0036
Epoch 11/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0050 - val_loss: 0.0034
Epoch 12/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step

In [13]:
# Predictions
def predict_all(model, data, input_length, output_length):
    predictions = []
    for i in range(len(data) - input_length + 1):
        input_data = data[i:i+input_length].reshape(1, input_length, n_features)
        pred = model.predict(input_data).reshape(output_length)
        predictions.append(pred)
    return np.array(predictions)

scaled_preds = predict_all(model, scaled_df, input_length, output_length)
preds = scaler.inverse_transform(scaled_preds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [21]:
import plotly.graph_objects as go

candles_back = 300
lpadding = 100
rpadding = 100

rlim = rpadding-candles_back if candles_back > rpadding else None

df_display = df.iloc[-lpadding-output_length-candles_back:rlim]

fig = go.Figure(data=[
    go.Candlestick(
        x=df_display.index,
        open=df_display['Open'],
        high=df_display['High'],
        low=df_display['Low'],
        close=df_display['Close']
    )
])

fig.add_trace(go.Scatter(
    x=df_display.index[lpadding:lpadding+output_length],
    y=preds[-output_length-candles_back-1],
    line_shape='hv',
    mode='lines'
))

fig.add_vrect(
    x0=df_display.index[lpadding],
    x1=df_display.index[lpadding+output_length-1],
    annotation_text="FORECAST",
    annotation_position="top left",
    fillcolor="green",
    opacity=0.2,
    line_width=0
)

fig.show()